# 信義路

In [1]:
import os, sys, copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import laspy
from tqdm.notebook import tqdm
from numba import jit, njit, prange
from glob import glob

In [2]:
las_folder = r"P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS/"
fn_list = glob(las_folder+"*.las")
fn = fn_list[0]
print(fn, os.path.basename(fn))

P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\001.las 001.las


In [3]:
@njit(parallel=True)
def get_upper_bound(points, min_points_num, upper_bound):
    points_max = np.max(points)
    points_min = np.min(points)
    for i in prange(np.int(points_max-points_min)):
        if np.sum(points_max-i >= points) < min_points_num:
            return upper_bound
        else:
            upper_bound = points_max-i

@njit(parallel=True)
def get_lower_bound(points, min_points_num, lower_bound):
    points_max = np.max(points)
    points_min = np.min(points)
    for i in prange(np.int(points_max-points_min)):
        if np.sum(points_min+i <= points) < min_points_num:
            return lower_bound
        else:
            lower_bound = points_min+i

In [4]:
%time
for fn in tqdm(fn_list):
    print(fn)
    inFile = laspy.file.File(os.path.join(las_folder, fn), mode='rw') # read a las file
    las_points = inFile.points.copy()
    min_points_num = int(inFile.x.shape[0]*.999)
    x_upper_bound, y_upper_bound, z_upper_bound = inFile.header.max
    x_lower_bound, y_lower_bound, z_lower_bound = inFile.header.min
    x_upper_bound_new = get_upper_bound(inFile.x, min_points_num, x_upper_bound)
    x_lower_bound_new = get_lower_bound(inFile.x, min_points_num, x_lower_bound)
    y_upper_bound_new = get_upper_bound(inFile.y, min_points_num, y_upper_bound)
    y_lower_bound_new = get_lower_bound(inFile.y, min_points_num, y_lower_bound)
    z_upper_bound_new = get_upper_bound(inFile.z, min_points_num, z_upper_bound)
    z_lower_bound_new = get_lower_bound(inFile.z, min_points_num, z_lower_bound)
    X_valid = (x_lower_bound_new <= inFile.x) * (x_upper_bound_new >= inFile.x)
    Y_valid = (y_lower_bound_new <= inFile.y) * (y_upper_bound_new >= inFile.y)
    Z_valid = (z_lower_bound_new <= inFile.z) * (z_upper_bound_new >= inFile.z)
    good_indices = X_valid * Y_valid * Z_valid
    good_points = las_points[good_indices]
    print(f"Original numbers of pts = {las_points.shape[0]}, numbers of filtering pts = {good_points.shape[0]}, ratio = {good_points.shape[0]/las_points.shape[0]}")
    inFile.header.max = [x_upper_bound_new, y_upper_bound_new, z_upper_bound_new] 
    inFile.header.min = [x_lower_bound_new, y_lower_bound_new, z_lower_bound_new] 
    outFile = laspy.file.File(f"P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_過濾極值點_LAS/{os.path.basename(fn)}", mode='w', header=inFile.header)
    outFile.points =  inFile.points[good_indices] 
    outFile.close() 
    inFile.close()

Wall time: 0 ns


P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\001.las
Original numbers of pts = 4588024, numbers of filtering pts = 4578749, ratio = 0.9979784325452526
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\002.las
Original numbers of pts = 8760640, numbers of filtering pts = 8748964, ratio = 0.9986672206596778
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\003.las
Original numbers of pts = 4135362, numbers of filtering pts = 4131255, ratio = 0.9990068584080426
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\004.las
Original numbers of pts = 46830917, numbers of filtering pts = 46745023, ratio = 0.9981658697821356
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\005.las
Original numbers of pts = 3093722, numbers of filtering pts = 3086578, ratio = 0.9976908073834688
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\006.las
Original numbers of pts = 62624, numbers of filtering pts = 62497, ratio = 0.9979720235053654
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\007.las
Original numbers of pts = 133

Original numbers of pts = 105060981, numbers of filtering pts = 104952594, ratio = 0.9989683420146248
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\055.las
Original numbers of pts = 24766385, numbers of filtering pts = 24743149, ratio = 0.9990617928292724
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\056.las
Original numbers of pts = 99785, numbers of filtering pts = 99686, ratio = 0.9990078669138648
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\057.las
Original numbers of pts = 3207563, numbers of filtering pts = 3204944, ratio = 0.9991834922649999
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\058.las
Original numbers of pts = 65497807, numbers of filtering pts = 65431097, ratio = 0.9989814926169971
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\059-1_0814.las
Original numbers of pts = 147861551, numbers of filtering pts = 147677328, ratio = 0.9987540844881304
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\059-2_0814.las
Original numbers of pts = 175943173, numbers of filtering pt

P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\106.las
Original numbers of pts = 1213265, numbers of filtering pts = 1211324, ratio = 0.9984001846257825
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\107.las
Original numbers of pts = 2353340, numbers of filtering pts = 2350347, ratio = 0.9987281905716981
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\108.las
Original numbers of pts = 83050396, numbers of filtering pts = 82918543, ratio = 0.9984123736146905
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\109.las
Original numbers of pts = 6779764, numbers of filtering pts = 6768511, ratio = 0.9983402077122449
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\110.las
Original numbers of pts = 138866, numbers of filtering pts = 138740, ratio = 0.9990926504687972
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\111.las
Original numbers of pts = 1530082, numbers of filtering pts = 1529952, ratio = 0.9999150372332986
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\112.las
Original numbers of pts = 1

Original numbers of pts = 2556054, numbers of filtering pts = 2553559, ratio = 0.9990238860368365
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\160.las
Original numbers of pts = 1609797, numbers of filtering pts = 1607432, ratio = 0.9985308706625742
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\161.las
Original numbers of pts = 656646, numbers of filtering pts = 654996, ratio = 0.9974872305625863
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\162.las
Original numbers of pts = 177152, numbers of filtering pts = 176815, ratio = 0.9980976788294798
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\163.las
Original numbers of pts = 933867, numbers of filtering pts = 931908, ratio = 0.9979022708801146
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\164.las
Original numbers of pts = 1581729, numbers of filtering pts = 1576135, ratio = 0.9964633638252823
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\165.las
Original numbers of pts = 1074901, numbers of filtering pts = 1072535, ratio = 0.9977988

P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\213.las
Original numbers of pts = 55482716, numbers of filtering pts = 55372382, ratio = 0.9980113806973689
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\214.las
Original numbers of pts = 50967627, numbers of filtering pts = 50822696, ratio = 0.9971564106761337
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\215.las
Original numbers of pts = 52853564, numbers of filtering pts = 52701853, ratio = 0.997129597542372
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\216.las
Original numbers of pts = 46716187, numbers of filtering pts = 46573249, ratio = 0.9969402896687608
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\217.las
Original numbers of pts = 42905011, numbers of filtering pts = 42828416, ratio = 0.9982147772902331
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\218.las
Original numbers of pts = 62682984, numbers of filtering pts = 62591096, ratio = 0.9985340838272792
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\219.las
Original numbers

P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\267.las
Original numbers of pts = 1217492, numbers of filtering pts = 1215831, ratio = 0.9986357199883038
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\268.las
Original numbers of pts = 445340, numbers of filtering pts = 444586, ratio = 0.9983069115731801
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\269.las
Original numbers of pts = 1265496, numbers of filtering pts = 1262734, ratio = 0.9978174565545841
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\270.las
Original numbers of pts = 1471316, numbers of filtering pts = 1467994, ratio = 0.9977421573611651
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\271.las
Original numbers of pts = 1342304, numbers of filtering pts = 1339378, ratio = 0.9978201659236656
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\272.las
Original numbers of pts = 1146934, numbers of filtering pts = 1142569, ratio = 0.9961942012356422
P:/HDmap/Close_Field/LiDAR/RS2003_Xinyi/點雲_LAS\273.las
Original numbers of pts = 280

# 青埔外圍

In [1]:
import os, sys, copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import laspy
from tqdm.notebook import tqdm
from numba import jit, njit, prange
from glob import glob

In [2]:
las_folder = r"P:/HDmap/Open_Field/LiDAR/QinPu/original/"
fn_list = glob(las_folder+"*.las")
fn_list_tmp = []
for i in fn_list:
    if not "ground" in i:
        fn_list_tmp.append(i)
fn_list = fn_list_tmp
fn = fn_list[0]
print(fn, os.path.basename(fn))

P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100000H.las 桃園青埔_6351020100000H.las


In [3]:
@njit(parallel=True)
def get_upper_bound(points, min_points_num, upper_bound):
    points_max = np.max(points)
    points_min = np.min(points)
    for i in prange(np.int(points_max-points_min)):
        if np.sum(points_max-i >= points) < min_points_num:
            return upper_bound
        else:
            upper_bound = points_max-i

@njit(parallel=True)
def get_lower_bound(points, min_points_num, lower_bound):
    points_max = np.max(points)
    points_min = np.min(points)
    for i in prange(np.int(points_max-points_min)):
        if np.sum(points_min+i <= points) < min_points_num:
            return lower_bound
        else:
            lower_bound = points_min+i

In [4]:
%time
for fn in tqdm(fn_list):
    print(fn)
    inFile = laspy.file.File(os.path.join(las_folder, fn), mode='rw') # read a las file
    las_points = inFile.points.copy()
    min_points_num = int(inFile.x.shape[0]*.999)
    x_upper_bound, y_upper_bound, z_upper_bound = inFile.header.max
    x_lower_bound, y_lower_bound, z_lower_bound = inFile.header.min
    x_upper_bound_new = get_upper_bound(inFile.x, min_points_num, x_upper_bound)
    x_lower_bound_new = get_lower_bound(inFile.x, min_points_num, x_lower_bound)
    y_upper_bound_new = get_upper_bound(inFile.y, min_points_num, y_upper_bound)
    y_lower_bound_new = get_lower_bound(inFile.y, min_points_num, y_lower_bound)
    z_upper_bound_new = get_upper_bound(inFile.z, min_points_num, z_upper_bound)
    z_lower_bound_new = get_lower_bound(inFile.z, min_points_num, z_lower_bound)
    X_valid = (x_lower_bound_new <= inFile.x) * (x_upper_bound_new >= inFile.x)
    Y_valid = (y_lower_bound_new <= inFile.y) * (y_upper_bound_new >= inFile.y)
    Z_valid = (z_lower_bound_new <= inFile.z) * (z_upper_bound_new >= inFile.z)
    good_indices = X_valid * Y_valid * Z_valid
    good_points = las_points[good_indices]
    print(f"Original numbers of pts = {las_points.shape[0]}, numbers of filtering pts = {good_points.shape[0]}, ratio = {good_points.shape[0]/las_points.shape[0]}")
    inFile.header.max = [x_upper_bound_new, y_upper_bound_new, z_upper_bound_new] 
    inFile.header.min = [x_lower_bound_new, y_lower_bound_new, z_lower_bound_new] 
    outFile = laspy.file.File(f"P:/HDmap/Open_Field/LiDAR/QinPu/點雲_過濾極值點_LAS/{os.path.basename(fn)}", mode='w', header=inFile.header)
    outFile.points =  inFile.points[good_indices] 
    outFile.close() 
    inFile.close()

Wall time: 0 ns


P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100000H.las
Original numbers of pts = 50411842, numbers of filtering pts = 50305415, ratio = 0.9978888492112628
P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100000H_1.las
Original numbers of pts = 32657223, numbers of filtering pts = 32591414, ratio = 0.9979848562138918
P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100000H_2.las
Original numbers of pts = 209254123, numbers of filtering pts = 208697363, ratio = 0.9973393116846735
P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100010H.las
Original numbers of pts = 49896904, numbers of filtering pts = 49762473, ratio = 0.9973058248263259
P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351020100010H_1.las
Original numbers of pts = 47254162, numbers of filtering pts = 47131292, ratio = 0.9973998057567924
P:/HDmap/Open_Field/LiDAR/QinPu/original\桃園青埔_6351320700000H.las
Original numbers of pts = 58946238, numbers of filtering pts = 58785723, ratio = 0.9972769254587545
P:

MemoryError: 

In [5]:
fn_list

['P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351020100000H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351020100000H_1.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351020100000H_2.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351020100010H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351020100010H_1.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351320700000H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351330700000H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351330700010H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351330700010H_1.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351330700020H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6351740100000H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6371590100000H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6371590100010H.las',
 'P:/HDmap/Open_Field/LiDAR/QinPu/original\\桃園青埔_6371600100000H.las',
 'P:/HDmap/O